In [1]:
# import modules
import pandas as pd
import numpy as np
import subprocess
import re

In [2]:
with open('best_params_PW.txt', 'r') as f:
    best_params = f.read()

best_params = eval(best_params)

# best parameters 출력
print(f"Best parameters from Optuna study:")
for param_name, value in best_params.items():
    print(f"{param_name}: {value}")


Best parameters from Optuna study:
p0: -0.2940311015411438
p1: -0.025274009809118717
p2: 0.5752564937514517
p3: 0.0149303259119247
p4: -0.031917239114805124
p5: 0.37429706351047715
p6: 0.17597827934796967
p7: 0.0019236165948633352
p8: -0.5325370067138747
p9: -0.25320551706017386
p10: -0.42394438144852936
p11: 0.4122796928843178
p12: -0.2824233218131415
p13: 0.24231864317993923
p14: 0.07933505414937016
p15: -0.0854332509114458
p16: -0.15035980075577965
p17: -0.3373248174236433
p18: 0.1342896622058419
p19: 0.1841645878417319
p20: 0.49791343111790326
p21: 0.7186410342672374
p22: 0.27286511521894063
p23: 0.3267625851774331
p24: -0.4214809883825277
p25: -0.05961369728692573
p26: 0.22141113837878806
p27: 0.4655246879118415
p28: -0.5614032252252504
p29: 0.24917739591414964
p30: 0.14711545237676518
p31: -0.6834981194419499
p32: -0.04770322439770082
p33: -0.048087282517392874
p34: 0.1418858536537195
p35: 0.3561357565700028
p36: 0.21378919031128876
p37: -0.08194466567472718
p38: 0.13392134693336

In [3]:
def modify_parameter(param_name: str, new_value: float, Voltage: float) -> None:
    # modify the parameter value in the kinet.inp file
    with open(f'kinet{Voltage}_opt.inp', 'r') as f:
        content = f.read()

    # modify the parameter
    pattern = rf'(parameter :: {param_name} = )([\d\.\+\-d]+)'
    new_value_str = f'{new_value:.4e}'.replace('e', 'd')
    content = re.sub(pattern, f'parameter :: {param_name} = {new_value_str}', content)
    
    with open(f'kinet{Voltage}_opt.inp', 'w') as f:
        f.write(content)

In [4]:
def run_prep(Voltage):
    kinet_path = f'kinet{Voltage}_opt.inp'
    process = subprocess.Popen(
        './preprocessor.exe',
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=False)
    
    process.stdin.write(f'{kinet_path}\n'.encode())
    process.stdin.flush()

    process.stdin.write('.\n'.encode())
    process.stdin.flush()

    output, error = process.communicate()
    output_str = output.decode('utf-8', errors='ignore') if output else ''
    error_str = error.decode('utf-8', errors='ignore') if error else ''

    return output_str, error_str

def compile_zdp(Voltage):
    exe_path = f'run_{Voltage}kV.exe'
    compile_command= [
        'gfortran', '-o', exe_path, 'dvode_f90_m.F90', 'zdplaskin_m.F90', f'run_{Voltage}kV.F90', 'bolsig_x86_64_g.dll'
    ]
    result = subprocess.run(compile_command, capture_output=True, text=True)

    if result.returncode != 0:
        raise Exception(f"{exe_path} 컴파일 실패")

def run_zdplaskin(Voltage):
    exe_path = f'run_{Voltage}kV.exe'
    try:
        process = subprocess.Popen(
            exe_path,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            stdin=subprocess.PIPE,
            universal_newlines=True,
            bufsize=1,
        )

        while True:
            output = process.stdout.readline()

            if not output:
                break
            print(f'\r{output.strip()}                                                                              ',end='',flush=True)

            if "PRESS ENTER TO EXIT" in output:
                process.kill()
                break

            if "WARNING: BOLSIG+ convergence failed" in output:
                process.stdin.write('\n')
                process.stdin.flush()

    except:
        pass
    return process

def err_cal(Voltage):
    #read the result file
    species = []
    with open('qt_species_list.txt', 'r') as f:
        for line in f:
            comp = line[2:]
            species.append(comp.strip())
    
    df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
    
    # calculate the concentration
    H2 = (df_sp['H2'])
    CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])
    C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V2)'] + df_sp['C2H2(V5)'] + df_sp['C2H2(V13)'])
    C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)'] + df_sp['C2H4(V2)'])
    C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)'] + df_sp['C2H6(V24)'])
    C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])
    C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])
    C4H10 = (df_sp['C4H9H'])
    C5H12 = (df_sp['C5H12'])
    C = (df_sp['C'])
    CH2 = (df_sp['CH2'])
    CH3 = (df_sp['CH3'])
    C2H3 = (df_sp['C2H3'])
    C2H5 = (df_sp['C2H5'])
    C3H7 = (df_sp['C3H7'])
    H = (df_sp['H'])
    CH3_plus = (df_sp['CH3^+'])
    CH4_plus = df_sp['CH4^+']
    CH5_plus = df_sp['CH5^+']
    C2H2_plus = df_sp['C2H2^+']
    C2H4_plus = df_sp['C2H4^+']
    C2H5_plus = df_sp['C2H5^+']
    C2H6_plus = df_sp['C2H6^+']
    C3H6_plus = df_sp['C3H6^+']
    C3H8_plus = df_sp['C3H8^+']

    t = abs(df_sp['Time [s]']-16.9646).argmin()

    sim_H2 = float(format(H2.iloc[t], '.6f'))
    sim_CH4 = float(format(CH4.iloc[t], '.6f'))
    sim_C2H2 = float(format(C2H2.iloc[t], '.6f'))
    sim_C2H4 = float(format(C2H4.iloc[t], '.6f'))
    sim_C2H6 = float(format(C2H6.iloc[t], '.6f'))
    sim_C3H6 = float(format(C3H6.iloc[t], '.6f'))
    sim_C3H8 = float(format(C3H8.iloc[t], '.6f'))
    sim_C4H10 = float(format(C4H10.iloc[t], '.6f'))
    sim_C5H12 = float(format(C5H12.iloc[t], '.6f'))
    sim_C = float(format(C.iloc[t], '.6f'))
    sim_CH2 = float(format(CH2.iloc[t], '.6f'))
    sim_CH3 = float(format(CH3.iloc[t], '.6f'))
    sim_C2H3 = float(format(C2H3.iloc[t], '.6f'))
    sim_C2H5 = float(format(C2H5.iloc[t], '.6f'))
    sim_C3H7 = float(format(C3H7.iloc[t], '.6f'))
    sim_H = float(format(H.iloc[t], '.6f'))
    sim_CH3_plus = float(format(CH3_plus.iloc[t], '.6f'))
    sim_CH4_plus = float(format(CH4_plus.iloc[t], '.6f'))
    sim_CH5_plus = float(format(CH5_plus.iloc[t], '.6f'))
    sim_C2H2_plus = float(format(C2H2_plus.iloc[t], '.6f'))
    sim_C2H4_plus = float(format(C2H4_plus.iloc[t], '.6f'))
    sim_C2H5_plus = float(format(C2H5_plus.iloc[t], '.6f'))
    sim_C2H6_plus = float(format(C2H6_plus.iloc[t], '.6f'))
    sim_C3H6_plus = float(format(C3H6_plus.iloc[t], '.6f'))
    sim_C3H8_plus = float(format(C3H8_plus.iloc[t], '.6f'))

    out_H2 = sim_H2 - sim_CH2 - 0.5*sim_CH3 - 0.5*sim_CH3_plus + 0.5*sim_CH5_plus - 0.5*sim_C2H3 + 0.5*sim_C2H5 + 0.5*sim_C2H5_plus + 0.5*sim_C3H7 + 0.5*sim_H
    out_CH4 = sim_CH4 + sim_CH2 + sim_CH3 + sim_CH3_plus + sim_CH4_plus + sim_CH5_plus
    out_C2H6 = sim_C2H6 + sim_C2H6_plus
    out_C2H4 = sim_C2H4 + sim_C2H3 + sim_C2H5 + sim_C2H4_plus + sim_C2H5_plus
    out_C2H2 = sim_C2H2 + sim_C2H2_plus
    out_C3H8 = sim_C3H8 + sim_C3H8_plus
    out_C3H6 = sim_C3H6 + sim_C3H7 + sim_C3H6_plus
    out_C4H10 = sim_C4H10
    out_C5H12 = sim_C5H12
    out_C = sim_C
    out_total = out_H2 + out_CH4 + out_C2H6 + out_C2H4 + out_C2H2 + out_C3H8 + out_C3H6 + out_C4H10 + out_C5H12 + out_C
    newsim = [out_H2/out_total*100, out_CH4/out_total*100, out_C2H6/out_total*100, out_C2H4/out_total*100, out_C2H2/out_total*100, out_C3H8/out_total*100, out_C3H6/out_total*100, out_C4H10/out_total*100, out_C5H12/out_total*100, out_C/out_total*100]
    
    return newsim

In [5]:
PD0 = 4.317
volt = [10, 12.5, 15]
res = []
for v in volt:
    if v == 10:
        PD = 2.64
    elif v == 12.5:
        PD = 3.273
    else:
        PD = PD0
    
    para = best_params.copy()
    for para_name in para.keys():
        para[para_name] = np.exp(best_params[para_name]*(PD0-PD))

    for para_name, value in para.items():
        modify_parameter(para_name, value, v)
    
    run_prep(v)
    compile_zdp(v)
    run_zdplaskin(v)

    sim = err_cal(v)
    res.append(sim)


PRESS ENTER TO EXIT ...At line 89 of file run_15kV.F90 (unit = 5, file = 'fort.5')                                                                                     

In [6]:
# load kinetic data
df_kinetic = pd.read_csv('kinetic_data.csv')
df_mol = pd.read_csv('kinetic_data.csv').filter(like='mol%')
df_power = pd.read_csv('kinetic_data.csv').filter(like='Power')

In [7]:
exp = df_mol.T
df_res = exp.copy()
df_res.columns = ['15kV_exp', '12.5kV_exp', '10kV_exp']
df_res['10kV_sim'] = res[0]
df_res['12.5kV_sim'] = res[1]
df_res['15kV_sim'] = res[2]
# 열 순서를 15kV, 12.5kV, 10kV 순서로 변경
df_res = df_res[['10kV_exp','10kV_sim', '12.5kV_exp', '12.5kV_sim', '15kV_exp', '15kV_sim']]








In [8]:
df_res

,10kV_exp,10kV_sim,12.5kV_exp,12.5kV_sim,15kV_exp,15kV_sim
mol%H2,4.076270,4.409488,6.680352,6.164690,9.803914,9.729240
mol%CH4,93.409239,92.822372,89.250216,90.020155,84.303804,84.466546
mol%C2H6,1.206263,1.337173,1.859689,1.796633,2.551080,2.550592
mol%C2H4,0.121577,0.133759,0.147585,0.135525,0.174030,0.169714
mol%C2H2,0.086712,0.123848,0.110815,0.143135,0.141543,0.181273
mol%C3H8,0.320123,0.322869,0.534193,0.488226,0.767533,0.805646
mol%C3H6,0.021062,0.022687,0.031208,0.030173,0.042723,0.049337
mol%C4H10,0.042958,0.034474,0.080897,0.052117,0.124365,0.055196
mol%C5H12,0.051410,0.014710,0.089128,0.052840,0.130899,0.192864
mol%C,0.664387,0.778620,1.215916,1.116506,1.960111,1.799593
